#  Loading the Category Data

The Yelp categories used to classify each business as to what it offers to consumers are based on a 
controlled vocabulary provided by Yelp.  In contrast to other fields in the business data which are free text
such that a business or reviewer adding a business can enter any value (e.g., city name), for categories a business
can only select from a predefined list.  Since it is a fixed list, it is referred 
to as a "controlled vocabulary" because the values anyone can enter are controlled.

The category vocabulary is provided at [this URL](https://www.yelp.com/developers/documentation/v3/all_category_list) which 
is part of Yelp's Fusion API (that apps can use to access Yelp data).  You will see that there are some categories that 
are at the top level (such as *Active Life* which starts the list) and others are indented as child categories (such as
*Amusement Parks* which is indented under Active Life).  

A list of all of the categories and their parent-child relationships can be downloaded from the above linked page, but
instead of the format used for the Yelp data, this file has JSON in a pretty printed format, so the JSON entry
for each category spans multiple lines.  In Spark version 2.2.1 there is a multi-line option that would allow us to read 
this directly, but currently we are using Spark 2.1.  Instead, we converted the file to a single line for each 
category as a JSON object and loaded that into DSX.

In the following cell, insert the boilerplate for the categories.json file (zipped as a .bz2 file) by selecting 
`Insert SparkSession DataFrame` from the `Insert to code` option for that file.  

In [122]:
# The code was removed by DSX for sharing.

### Creating a DataFrame
After you insert the bolerplate code above, move the line for creating the 
df_data_1 DataFrame to the blank cell below this markdown cell.  Un-comment the line (remove the # sign at the 
start of the line) and change the name of the DataFrame variable to `df_categories`.  

You can delete the commented out line to `take` 5 rows.  If your bolerplate code instead just defines the path_1 variable with the URL to your file, move
that line to the cell below, change the variable name to `path_categories` and add the following line:

`
df_categories = spark.read.json(path_categories)
`

Also add lines to print out the total number of category records in the DataFrame, print the schema that was inferred by Spark, and show the first 20 rows (using `truncate=False`).

In [123]:
path_categories = cos.url('categories.json.bz2', 'spring2018andy023363be332e40639c4287c87e0af5e0')
df_categories = spark.read.json(path_categories)


df_categories = spark.read.json(path_categories)
print "Number of Categories: ", df_categories.count()

df_categories.printSchema()


Number of Categories:  1539
root
 |-- alias: string (nullable = true)
 |-- country_blacklist: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- country_whitelist: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- parents: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)



### What the Category Data Means

Following is a description of the fields in the categories data:
* alias:  This is similar to the `title` field, but it's all lower case and there are no spaces in the name.  This is also the value used in the `parents` field to identify the parent-child relationships in the categry hierarchy.
* country_blacklist:  A category may have a white list or black list, but not both.  This field is either a list of 2-digit country codes or null. if this field contains a list of country codes, then the category can be used in any country except those in the list.  For example, the cuisine "Afghan" has the blacklist \[MX, TR\], which means this category is not used in Mexico or Turkey.
* country_whitelist:  If there is a list in this field, the category only is used in those countries.  For example, the category "Absinthe Bars", which is a sub-category under "Bars" has a whitelist with one country \[CZ\] which is the Czech Republic.  This category will only be used to describe businesses in that country.
* parents: The categories form a hierarchy, so the broader parent category above each category is a list in this field.  although most categories have one parent category, there are a few with multiple parent categories in the list.  If the list is empty \[ \] for a category (such as "Active Life"), then the category is a top-level category and there is no parent category.
* title:  This is the human-readable label for the category and is also the label included in the list of categories for a business in the `categories` field in the Yelp business data.




### Creating the Temporary View

Run the cell below to create a temporary view for your DataFrame.  It will run fast - remember Spark is lazy and you have not yet told it to do anything with this temporary view.

In [124]:
df_categories.createOrReplaceTempView("categories")

# Profiling the Category Data

We need to start learning about our data:
* How many categories have multiple parents?  For those that do, how many parents do they have?
* We want to determine the hierarchy for the categories - how deep does it go?
* What are the top-level categories and how many are there? 
* How many categories have a whitelist and how many include the U.S. or Canada?
* How many categories have a blacklist, and how many include the U.S., or Canada (and how many include both)?

### Multiple parent categories
We will start our profiling with determining the number of categories with multiple parents.

In relational databases values should be atomic.  What this means is that the value for each column
in a row should contain a single value.  For example, if categories had multiple parents, there would be a separate table that 
contains that relationship.  Why not just have another row?  Database normalization - you want to avoid duplicate values.
In a relational world, this is important.  In part this is to maintain data integrity - if you have multiple rows for a category
because it has multiple parents, and you update the alias for one row and not the other, now you have a problem.

In Big Data (such as on Spark), columns can contain arrays (also known as lists), or maps (also known as dictionaries, or objects in JSON).
Similar to functions you can use with numeric or string columns in a DataFrame/temporary view, there are functions you can use in your SQL queries
with array and map fields.  One of these functions is `SIZE()`, which returns the size of an array.  For the parents field, we are most interested
(at least in our initial profiling) in those categories with multiple parents or no parents.

Run the following cell to find out how many categories have multiple parents.  Since we first just want to know the answer, 
there is no need to save the DataFrame generated (which in this case will be a DataFrame with 1 column and one row).

In [125]:
spark.sql("""
SELECT COUNT(alias) AS category_count
FROM categories
WHERE SIZE(parents) > 1
""").show()

+--------------+
|category_count|
+--------------+
|            16|
+--------------+



Not too many categories have multiple parents, only 16 out of a total of 1,539 categories in Yelp.
For now we are not going to do too much yet with these, but let's see the detail.  In the next cell (which is blank),
Add a SQL query to show the detail for all of these 16 categories.  Also, add a column to your query to get the number of parents for each of these 16 categories.

You should find that 15 of the 16 categories have 2 parents and 1 category has 3 parents.  In the `show()` function, include `truncate=False` to be able to see the parent category names.

In [126]:
spark.sql("""
SELECT alias AS Alias, SIZE(parents) AS Parents
FROM categories
WHERE SIZE(parents) > 1
ORDER BY parents DESC
""").show(truncate=False)

+---------------+-------+
|Alias          |Parents|
+---------------+-------+
|stationery     |3      |
|cosmetics      |2      |
|eltern_cafes   |2      |
|deptstores     |2      |
|isps           |2      |
|galleries      |2      |
|hair_extensions|2      |
|hotels         |2      |
|wineries       |2      |
|jazzandblues   |2      |
|kiosk          |2      |
|medicalspa     |2      |
|musicvenues    |2      |
|perfume        |2      |
|sportswear     |2      |
|swimminglessons|2      |
+---------------+-------+



### Top Level Categories

Since the categories form a hierarchy, we need to find out how many categories there are at the top of this hierarchy.  In your earlier queries, you saw that the 
"Active Life" category (alias `active`) had an empty list for its parents, so it is at the top of the hirearchy.  In the following cell, write a query to get all of the 
categories at the top level using the `SIZE()` function (hint: the size of the parent list for the Active Life category is 0).  Keep in mind that if your result 
says "Only showing top 20 rows", then you aren't seeing all of the categories returned by your query.

In [127]:
spark.sql("""
SELECT alias AS Alias, SIZE(parents) as Parents
FROM categories
WHERE SIZE(parents) = 0
""").show(30, truncate=False)

+------------------+-------+
|Alias             |Parents|
+------------------+-------+
|active            |0      |
|arts              |0      |
|auto              |0      |
|beautysvc         |0      |
|bicycles          |0      |
|education         |0      |
|eventservices     |0      |
|financialservices |0      |
|food              |0      |
|health            |0      |
|homeservices      |0      |
|hotelstravel      |0      |
|localflavor       |0      |
|localservices     |0      |
|massmedia         |0      |
|nightlife         |0      |
|pets              |0      |
|professional      |0      |
|publicservicesgovt|0      |
|religiousorgs     |0      |
|restaurants       |0      |
|shopping          |0      |
+------------------+-------+



# Business Categories

We will return to categories to determine the rest of the hierarchy and how deep it is, but first we are going to turn to the business data
and determine the categories each business is in.  One reason for doing this is that we may want to know how often the categories with
multiple parents are used, that could inform how much effort is put into deciding how to handle categories with multiple parents.

In the cell below, add the business.json.bz2 file that you zipped up and hoopefully loaded to your project earlier in the semester.
When adding the business data to this notebook, do the following:
* Load the data into a DataFrame named df_business
* Print the number of records in the data (how many businesses do we have data on?)
* Print the schema for the business data

In [128]:

# Please read the documentation of PySpark to learn more about the possibilities to load data files.
# PySpark documentation: https://spark.apache.org/docs/2.0.1/api/python/pyspark.sql.html#pyspark.sql.SparkSession
# The SparkSession object is already initialized for you.
# The following variable contains the path to your file on your IBM Cloud Object Storage.
path_2 = cos.url('business.json.bz2', 'spring2018andy023363be332e40639c4287c87e0af5e0')


In [129]:
business = cos.url('business.json.bz2', 'spring2018andy023363be332e40639c4287c87e0af5e0')

df_business = spark.read.json(business)


df_business = spark.read.json(business)
print "Number of Businesses: ", df_business.count()

df_business.printSchema()


Number of Businesses:  174567
root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: boolean (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: struct (nullable = true)
 |    |    |-- casual: boolean (nullable = true)
 |    |    |-- classy: boolean (nullable = true)
 |    |    |-- divey: boolean (nullable = true)
 |    |    |-- hipster: boolean (nullable = true)
 |    |    |-- intimate: boolean (nullable = true)
 |    |    |-- romantic: boolean (nullable = true)
 |    |    |-- touristy: boolean (nullable = true)
 |    |    |-- trendy: boolean (nullable = true)
 |    |    |-- upscale: boolean (nullable = true)
 |    |-- BYOB: boolean (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: struct (nullable = true)
 |    |    |-- friday: boolean (nullable = true)
 |    |    |-- monday: boolean (nullable = true)
 |    |   

### Filtering Out Columns

If you look at the above schema, you will see that most of the data about businesses are attributes - these may be very useful for your team's question, 
but for now we are going to focus on the categories each business is in.  For the categories data we created a temporary view, but for businesses we are going to 
first select out the columns we want from the DataFrame and then create a temporary view.  The columns we will select are:

* business_id (or we will never be able to join with the other Yelp data)
* name (a human readable name for us to see)
* state 
* city (we will generally be using the state for location)
* is_open (in case we decide it matters if they are still in business)
* review_count
* categories

In the following cell we do a select directly on the df_business DataFrame and then create a temporary view.  We also do a quick query 
to see a few rows as a sanity check to make sure nothing has gone horribly wrong.

In [130]:
df_business.select("business_id","name","state","city","is_open","review_count","categories").createOrReplaceTempView("business")

spark.sql("""
SELECT * 
FROM business
""").show(truncate=False)

+----------------------+--------------------------+-----+--------------+-------+------------+------------------------------------------------------------------------------------------------------+
|business_id           |name                      |state|city          |is_open|review_count|categories                                                                                            |
+----------------------+--------------------------+-----+--------------+-------+------------+------------------------------------------------------------------------------------------------------+
|FYWN1wneV18bWNgQjJ2GNg|Dental by Design          |AZ   |Ahwatukee     |1      |22          |[Dentists, General Dentistry, Health & Medical, Oral Surgeons, Cosmetic Dentists, Orthodontists]      |
|He-G7vWjzVUysIKrfNbPUQ|Stephen Szabo Salon       |PA   |McMurray      |1      |11          |[Hair Stylists, Hair Salons, Men's Hair Salons, Blow Dry/Out Services, Hair Extensions, Beauty & Spas]|
|KQPW8lFf1y5BT2

### Flagging Businesses as Being in Certain Categories

The categories for a business are an array in the business data.  Later we will figure out the hierachy of the categories so we can match them up with the business data, 
but we want to flatten the category data, so if a business has the following categories (such as Alize Catering in Toronto):

\[Italian, French, Restaurants\]

We want to see three rows in the result with all of the data repeated, but each of the categories on a separate line.

However, depending on our question, we may want to first flag certain businesses as being in a specific top-level category.  Why is this?  Well, if we flatten the data for Alize Catering
but then want to know if the business is a restaurant, it becomes a little more tricky.  If we add a new column to "flag" Alize Catering as a restaurant, then all three lines for Alize Catering
after we flatten the data will contain that same value.

In Yelp's 10K, Restaurants and Shopping are identified 
as the two most common categories, so lets flag if a business is in either of those categories.  We will also flag Nightlife and Food.  There are a number of ways we could do this, but we 
will add a column for each of these, so we will add four columns that have a Boolean value (True or False):

* is_shopping
* is_restaurants
* is_food
* is_nightlife

To do this, we will use another array function: `ARRAY_CONTAINS(<column>, <value>)`  Which returns a Boolean value. So to check for shopping we would include the following in our select:

`ARRAY_CONTAINS(categories, "Shopping") AS is_shopping`

Looking at the top categories we identified earlier, there are two that you may wonder if they ever occur by themselves or always with another top level category:

* Local Flavor
* Local Services

For these two categories, instead of having separate columns, let's have one column named "is_local" that has the following values:

* 0 = neither local flavor nor local service
* 1 = local flavor, but not local service
* 2 = local service, but not local flavor
* 3 = both local flavor and local service

We can use the `ARRAY_CONTAINS()` function, but it's getting a bit more complicated.  In a prior exercise we used the `IF()` function, but with our 4 possible values, 
that starts to get messy, so we will use the `CASE` function which works well for multiple conditions.  The syntax of the CASE function is as follows:

`
CASE
    WHEN ARRAY_CONTAINS(categories, "Local Flavor") AND ARRAY_CONTAINS(categories, "Local Services") THEN 3
    WHEN ARRAY_CONTAINS(categories, "Local Flavor") THEN 1
    WHEN ARRAY_CONTAINS(categories, "Local Services") THEN 2
    ELSE 0
END AS is_local
`

The entire `CASE` function, from the `CASE` at the start to the `END` is calculating the value of a single column in the `SELECT` clause.  We then give that column the alias "is_local".  
You may be wondering why the `THEN` results within the CASE function are not in numerical order, but the way this function works is that it starts at the top and keeps testing the `WHEN` conditions.
 When some condition is true, the corresponding `THEN` value is the result for that row of the data.  If none of the `WHEN` conditions is true, the `ELSE` value at the end is the default result.

### Let's put this all together

The query in the following cell includes the function calls for is_shopping and is_restaurants, but you need to add the two lines for the is_food and is_nightlife columns:

In [131]:
df_business_top_categories = spark.sql("""
SELECT *,
       ARRAY_CONTAINS(categories, "Shopping") AS is_shopping,
       ARRAY_CONTAINS(categories, "Restaurants") AS is_restaurants,


       CASE
           WHEN ARRAY_CONTAINS(categories, "Local Flavor") AND ARRAY_CONTAINS(categories, "Local Services") THEN 3
           WHEN ARRAY_CONTAINS(categories, "Local Flavors") THEN 1
           WHEN ARRAY_CONTAINS(categories, "Local Services") THEN 2
           ELSE 0
       END AS is_local 
FROM business
""")

df_business_top_categories.show()

+--------------------+--------------------+-----+--------------+-------+------------+--------------------+-----------+--------------+--------+
|         business_id|                name|state|          city|is_open|review_count|          categories|is_shopping|is_restaurants|is_local|
+--------------------+--------------------+-----+--------------+-------+------------+--------------------+-----------+--------------+--------+
|FYWN1wneV18bWNgQj...|    Dental by Design|   AZ|     Ahwatukee|      1|          22|[Dentists, Genera...|      false|         false|       0|
|He-G7vWjzVUysIKrf...| Stephen Szabo Salon|   PA|      McMurray|      1|          11|[Hair Stylists, H...|      false|         false|       0|
|KQPW8lFf1y5BT2Mxi...|Western Motor Veh...|   AZ|       Phoenix|      1|          18|[Departments of M...|      false|         false|       0|
|8DShNS-LuFqpEWIp0...|    Sports Authority|   AZ|         Tempe|      0|           9|[Sporting Goods, ...|       true|         false|       0|

### Flatten the categories

We now want to flatten the categories.  To do this there is a function named `EXPLODE()` that will take a column contianing an array or map and flatten it out to create new rows.
When flattening data using `EXPLODE()`, keep in mind that each query can only explode one column.

**NOTE:** When using `EXPLODE()`, if a row contains null in the exploded column, it **WILL NOT** be included in the result.  There is an `EXPLODE_OUTER()` function that will retain
the row with the original null value, but that was added in version 2.3 of Spark (and currently DSX is on version 2.1 of Spark).  Later in this notebook we show a way that could be used to get around this
by combining multiple SELECT clauses in a query using a UNION.

To flatten the categories field from the df_business_top_categories DataFrame you created above, we first need to create a new temporary view, which we name "business_cat" in the 
cell below.

#### What you need to add: 
In that cell, after the line that creates the new temporary view, add a query that selects each field from "business_cat" (these are the same columns that are in the DataFrame it's built on).
In this query you cannot use the asterisk \* to select all of the columns, but will instead need to list them.  The reason is that instead of the "categories" column you need to add the following 
`EXPLODE()` function call:

`
EXPLODE(categories) AS category
`

Assign the result of the query to a new DataFrame named `df_flattened_business`, and then show the DataFrame as you did above.

In [132]:
df_business_top_categories.createOrReplaceTempView("business_cat")

df_flattened_business = spark.sql("""
SELECT business_id, name, state, city, is_open, review_count, EXPLODE(categories) AS category,
       ARRAY_CONTAINS(categories,"Food") AS is_food,
       ARRAY_CONTAINS(categories, "Shopping") AS is_shopping,
       ARRAY_CONTAINS(categories, "Restaurants") AS is_restaurants,


       CASE
           WHEN ARRAY_CONTAINS(categories, "Local Flavor") AND ARRAY_CONTAINS(categories, "Local Services") THEN 3
           WHEN ARRAY_CONTAINS(categories, "Local Flavor") THEN 1
           WHEN ARRAY_CONTAINS(categories, "Local Services") THEN 2
           ELSE 0
       END AS is_local 
FROM business_cat
""")

df_flattened_business.show(truncate=False)



+----------------------+------------------------+-----+--------------+-------+------------+-----------------------------+-------+-----------+--------------+--------+
|business_id           |name                    |state|city          |is_open|review_count|category                     |is_food|is_shopping|is_restaurants|is_local|
+----------------------+------------------------+-----+--------------+-------+------------+-----------------------------+-------+-----------+--------------+--------+
|FYWN1wneV18bWNgQjJ2GNg|Dental by Design        |AZ   |Ahwatukee     |1      |22          |Dentists                     |false  |false      |false         |0       |
|FYWN1wneV18bWNgQjJ2GNg|Dental by Design        |AZ   |Ahwatukee     |1      |22          |General Dentistry            |false  |false      |false         |0       |
|FYWN1wneV18bWNgQjJ2GNg|Dental by Design        |AZ   |Ahwatukee     |1      |22          |Health & Medical             |false  |false      |false         |0       |
|FYW

# How Often Are Categories With Multiple Parents Used?

Now that we have the categories used at each business, let's look at how many times each of those 16 categories with multiple parents were actually used 
by a business in our data. To do this we are going to join the `categories` temporary view we created earlier with our business data.  First we need to 
create a temporary view for the df_flattened_business DataFrame.

#### In the cell below, create a new temporary view named "flattened_business" from that DataFrame

In [133]:
df_flattened_business.createOrReplaceTempView("flattened_business")

### Join the categories and businesses

In the following cell we join the "categories" temporary view defined earlier and your new "flattened_business" temporary view where the title column from the categories matches the 
category column from the flattened business data.  Since we only care about those categories with multiple parents (at least for the moment), we also need to include that criteria in
our query.

At the end of the cell we group our DataFrame based on the category column and get a count of how many times each of these multi-parent categories is used. Here we are profiling to learn how 
often each of these categories are used.  As it rurns out, some of these are fairly common (e.g., Hotels).

In [134]:
df_problem_categories = spark.sql("""
SELECT B.business_id, B.name, B.category, C.alias, C.parents
FROM flattened_business AS B INNER JOIN categories AS C
ON B.category = C.title
WHERE SIZE(C.parents) > 1
""")

df_problem_categories.show(truncate=False)

print "record count:", df_problem_categories.count()

df_problem_categories.groupBy("alias").count().orderBy("count", ascending=False).show(truncate=False)

+----------------------+---------------------------------------+-------------------------+---------------+-----------------------------+
|business_id           |name                                   |category                 |alias          |parents                      |
+----------------------+---------------------------------------+-------------------------+---------------+-----------------------------+
|He-G7vWjzVUysIKrfNbPUQ|Stephen Szabo Salon                    |Hair Extensions          |hair_extensions|[beautysvc, hair]            |
|lj0MiK5_fyv9df2twnsI7g|Pampered Hair Passionate about Hair    |Hair Extensions          |hair_extensions|[beautysvc, hair]            |
|NFTh6mj3X2AnHJCKIFUk5Q|Fairfield Inn & Suites                 |Hotels                   |hotels         |[eventservices, hotelstravel]|
|oB5KH-jYU93w-QnHas6EMA|Aces High Tobacco & Gifts              |Art Galleries            |galleries      |[shopping, arts]             |
|0953KSZ26LmJ7CDacVzpYg|dd's DISCOUNTS   

### Assigning parents

As you can see in the above summary, some of these multi-parent categories are used often, such as Hotels, Department Stores, Cosmetics & Beauty Supply, but even Medical Spas, and 
Hair Extensions are fairly frequent categories.  We need to assign single parents in these cases so we can build a hierarchy (referred to a tree, but obviously by someone who does 
not live near Yosemite since the tree is upside down).

For some of these 16 categories, a parent choice might be more obvious, such as for the "Hotels" category the two parents are "eventservices" and "hotelstravel".  In this case 
"hotelstravel" seems like it may be a more suitable parent.

Since we have only 16 of these multi-parent categories, we will create a new DataFrame named "df_single_parent" using a case statement.

In the following blank cell create a Spark SQL query that includes all of the columns in the categories temporary view you created earlier, but instead of the 
list currently in the parents column, your query result should have a "parent" column (not plural), that has the parent assigned to each category.  For the `SELECT` clause,
you should have the new "parent" column last instead of the "title" column.

You have 1,539 categories, but only 16 of them need a parent assigned, so the first `WHEN` condition in your case statement should check for the situation
where there is a single parent - that occurs when the SIZE(parents) is 1.  In that case, the `THEN` result for that case is to get the first item in that array.
The first element is in the 0th position, so the `THEN` for this case would be `parents[0]`

The second `WHEN` in your `CASE` function needs to address the top-level categories that have no parents, their "parents" array is empty: \[ \], so when 
the `SIZE(parents) = 0`  the `THEN` result is `NULL`, meaning there is no value:

`
WHEN SIZE(parents) = 0 THEN NULL
`

After you run your query, show a count of your DataFrame and show the first few rows.  
You should have the same number of rows you started with and the following columns (in this order):

alias, country_blacklist, country_whitelist, title, parent

After you have the query running, you may want to try adding the followng `WHERE` clause just to test your `CASE` function (but be sure to then remove 
the `WHERE` clause and rerun the query so the result has all 1,539 rows again).

`WHERE SIZE(parents) > 1`

In [135]:
df_single_parent = spark.sql("""
SELECT Alias, country_blacklist as Blacklist, country_whitelist as Whitelist, Title, 
       CASE
           WHEN SIZE(parents) = 1 THEN parents[0]
           WHEN SIZE(parents) = 0 THEN NULL
       END AS Parent
FROM categories
""")

df_single_parent.show(7, truncate=False)

print "Record count:", df_single_parent.count()
df_single_parent.groupBy("alias").count().orderBy("count", descending=True).show(5, truncate=False)

+-------------+------------------------+---------+--------------+-------------+
|Alias        |Blacklist               |Whitelist|Title         |Parent       |
+-------------+------------------------+---------+--------------+-------------+
|3dprinting   |null                    |null     |3D Printing   |localservices|
|abruzzese    |null                    |[IT]     |Abruzzese     |italian      |
|absinthebars |null                    |[CZ]     |Absinthe Bars |bars         |
|acaibowls    |[TR, MX, AR, CL, PL, IT]|null     |Acai Bowls    |food         |
|accessories  |null                    |null     |Accessories   |fashion      |
|accountants  |null                    |null     |Accountants   |professional |
|acnetreatment|null                    |null     |Acne Treatment|beautysvc    |
+-------------+------------------------+---------+--------------+-------------+
only showing top 7 rows

Record count: 1539
+---------------+-----+
|alias          |count|
+---------------+-----+
|fra

# Identifying the Hierarchy of Categories

For some project questions, you will want to be able to identify categories within a particular parent.  For example, all restaurant cuisine categories
are under the top-level "Restaurants" category in the Yelp hierarchy.  However, some cuisines break down into lower level sub-categories.  For example,
the category "Caribbean" has four subcategories: Dominican, Haitian, Puerto Rican, and Trinidadian.  If a restaurant has "Haitian" as a category, the 
Yelp data for that business will automatically include all of the parent categories up to the top, so it will have \[Haitian, Caribbean, Restaurants\].  If
you are splitting restaurants between cuisines, you will want to use the second level categories (e.g., Caribbean) and omit the lower level categories.

In the following cell we do a transformation that creates a new DataFrame named `df_category_hierarchy` that is the `df_single_parent` DataFrame with a `level`
column added that indicates how far down in the hierarchy each category is.  The top-level categories (such as Restaurants or Shopping) are `level` = 1.

#### What the followng cell is doing

Since DataFrames and temporary views based on them are immutable (cannot be updated), we do a loop through the ierarchy, starting at the top level
and adding each level as we identify the children categories of the last level added.

At the start, the `df_category_hierarchy` DataFrame is populated with the 22 top-level categories that do not have a parent (their parent is null).
In each loop through the while statement (which is plain old Python), the following happens:
* The current size of the hierachy is calculated as the variable `before`.  In the first loop this is 22, the number of top-level categories
* A temporary view named `hierarchy` is created based on the current `df_category_hierarchy` DataFrame
* The query is a UNION of two SELECT statements:
    1. The current content of the `df_category_hierarchy` DataFrame
    2. The INNER JOIN of rows in the hierarchy that are at the parent level (so we don't keep adding the levels already added) with the child categories that have those categories as their parent
* The result of the UNION is assigned to the `df_category_hierarchy` DataFrame and we calculate how many records were added
* We stop looping when no new records are added (it means we reached the bottom of the hierarchy)

In the UNION, the number of columns must match.  Conceptually,you can think of this as if you had a spreadsheet with the 22 rows containing the 
top-level categories and then you appended the rows for the second-level categories that had those top-level categories as parents, and then 
you appended rows for the next level, and so on.

In [136]:
df_single_parent.createOrReplaceTempView("parents")

In [137]:

df_category_hierarchy = spark.sql("""
SELECT *, 1 AS level, alias AS top_parent
FROM parents
WHERE parent IS NULL
""")
parent_level = 0
added = df_category_hierarchy.count()
print "First level of hierachy:", added

while added > 0:
    before = df_category_hierarchy.count()
    df_category_hierarchy.createOrReplaceTempView("hierarchy")
    parent_level = parent_level + 1
    
    query = "SELECT * FROM hierarchy " + \
            "UNION " + \
            "SELECT P.*, " + str(parent_level + 1) + " AS level, H.top_parent " + \
            "FROM hierarchy AS H INNER JOIN parents AS P " + \
            "ON H.alias = P.parent " + \
            "WHERE H.level = " + str(parent_level)
    
    df_category_hierarchy = spark.sql(query)
    added = df_category_hierarchy.count() - before
    print "number of records added at level", str(parent_level + 1), ":", added
    
print "record count in hierarchy:", df_category_hierarchy.count()
df_category_hierarchy.show(50,truncate=False)
    


First level of hierachy: 38
number of records added at level 2 : 972
number of records added at level 3 : 511
number of records added at level 4 : 18
number of records added at level 5 : 0
record count in hierarchy: 1539
+--------------------+------------------------------------------------+------------------------------------------------+------------------------------+-----------------------+-----+-------------+
|Alias               |Blacklist                                       |Whitelist                                       |Title                         |Parent                 |level|top_parent   |
+--------------------+------------------------------------------------+------------------------------------------------+------------------------------+-----------------------+-----+-------------+
|aviationservices    |null                                            |null                                            |Aviation Services             |auto                   |2    |auto      

# Do Some More Profiling!

In the empty cells below, use the temporary views and DataFrames you have created so far to answer the following profiling questions to learn about the data (and add markdown cells to say what is being done):

1. Under the top-level Restaurants category, how many cuisines are there at the level directly below (level 2)?
2. Are there more businesses that have the "Local Flavor" category that also have the "Shopping" category or the "Restaurants" category?
3. How many categories are whitelisted in the United States?

# Cuisines that are top-level and below (level 2)

The SQL query below is to count the amount of cuisines that are below level 2 from the temporary view hierarchy. Also the top_parent is established to specifically target restaurants and not any other categories such as education, health, or shopping. We set level = 2 and top_parent = "restaurants" which will simplify the search and specifically pull the needed data.

In [138]:
#Under the top-level Restaurants category, how many cuisines are there at the level directly below (level 2)?

print "Number of Cuisines: "

df_top_level = spark.sql("""
SELECT COUNT(title) as Number_Cuisines
FROM hierarchy
WHERE level = 2 AND top_parent = "restaurants"
""")

df_top_level.show(50, truncate=False)

Number of Cuisines: 
+---------------+
|Number_Cuisines|
+---------------+
|189            |
+---------------+



# Local Flavor with Shopping or Restaurants
The SQL query below is to discover how many businesses contain the categories local flavor, shopping, and restaurants. We set the function to df_local_flavor. To find this count we establish a CASE and WHEN statement which pull out any array that contains the key words. To simplify the results the count function is included on Local_Flavor to display the amount of businesses. 

In [139]:
#Are there more businesses that have the "Local Flavor" category that also have the "Shopping" category or the "Restaurants" category?

print "Business that have Local Flavor w/ Shopping or Restaurants: " 

df_local_flavor = spark.sql("""
SELECT business_id, name, state, city,
       CASE
           WHEN ARRAY_CONTAINS(categories, "Local Flavor") AND ARRAY_CONTAINS(categories, "Shopping") THEN "Shopping"
           WHEN ARRAY_CONTAINS(categories, "Local Flavor") AND ARRAY_CONTAINS(categories, "Restaurants") THEN "Restaurants"
           ELSE "Neither"
       END AS Local_Flavor 
FROM business
""")

df_local_flavor.groupBy('Local_Flavor').count().orderBy('count', ascending = False).show(10, truncate = False)


Business that have Local Flavor w/ Shopping or Restaurants: 
+------------+------+
|Local_Flavor|count |
+------------+------+
|Neither     |174253|
|Shopping    |162   |
|Restaurants |152   |
+------------+------+



# Whitelisted categories in the United States (U.S)
The query below required an explode feature to help break up the list of states that are currently whitelisted. It was impossible to pull the data by itself because of a syntax error. Breaking it up and adding a count was the best way to determine how many categories were in the United States. The best way to single out each category was with the EXPLODE function on whitelist. 

In [140]:
#How many categories are whitelisted in the United States?

df_whitelist = spark.sql("""
SELECT Alias, Title, EXPLODE(Whitelist) AS WhiteList, parent
FROM hierarchy
""")

print "Number of Whitelisted Categories in the US: "

df_whitelist.groupBy('Whitelist').count().orderBy("COUNT", ascending = False).show(1, truncate = False)

Number of Whitelisted Categories in the US: 
+---------+-----+
|Whitelist|count|
+---------+-----+
|US       |271  |
+---------+-----+
only showing top 1 row

